In [38]:
from interestrate_tariff import tariff_search, interestrate_search
from langchain_core.tools import tool
from typing import Optional
import asyncio
from typing import List, Dict
@tool
def general_economic_analyze(max_articles: int = 10) -> str:
    """
    경제시장에 대한 분석을 요청하면, 이 tool을 사용한다.
    Use this tool to analyze the general economic market.
    It will fetch the latest news titles and also crawl the article content (summary).
    This tool performs searches using the keyword "tarrif", allowing it to analyze the general economic market. 
    
    Useful when the user asks to:
    - "Tell me about the general economic market"
    - "현재 경제시장이 어때"
    - "금리 동향이 어때"
    
    Arguments:
        max_articles (int): Number of articles to retrieve.
    """
    pipeline = tariff_search(max_articles=max_articles)
    try:
        results = asyncio.run(pipeline.run_pipeline())
        if not results:
            return f"검색 결과가 없습니다."
        
        output = f"검색 기사 {len(results)}건:\n\n"
        for i, article in enumerate(results, 1):
            output += f"{i}. {article['title']}\n"
            output += f"URL: {article['url']}\n"
            output += f"본문 요약:\n{article['content'][:700]}...\n\n"
    except Exception as e:
        return f"오류 발생: {str(e)}"
    pipeline = interestrate_search(max_articles=max_articles)
    try:
        results = asyncio.run(pipeline.run_pipeline())
        if not results:
            return f"검색 결과가 없습니다."
        
        output += f"검색 기사 {len(results)}건:\n\n"
        for i, article in enumerate(results, 1):
            output += f"{i}. {article['title']}\n"
            output += f"URL: {article['url']}\n"
            output += f"본문 요약:\n{article['content'][:700]}...\n\n"
        return output

    except Exception as e:
        return f"오류 발생: {str(e)}"

In [39]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
os.environ["GOOGLE_API_KEY"] = '' # API KEY 입력
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [40]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate([
("system", """
다음은 경제 뉴스 기사 요약이다. 이를 종합하여 다음 기준에 따라 구체적으로 분석하라:

1. **금리 정책 동향**: 각국 중앙은행의 기준금리 변동, 인하/인상 가능성, 시장 반응 등 수치를 포함해 서술하라.
2. **관세 및 무역 정책**: 어떤 국가가 어느 국가에 대해 어떤 관세를 부과하거나 협상을 하고 있는지 구체적으로 기술하라. 적용 시기나 수치(예: 15~20% 관세)도 명확히 제시하라.
3. **자산 시장 반응**: 주가, 금, 원자재 등의 가격 변화(예: "금 0.4% 상승", "ASX 1% 상승")를 정량적으로 서술하라.
4. **환율 및 외환시장 변화**: 주요 통화 간 환율 변화와 그 원인을 구체적으로 정리하라.
5. **AI, 반도체, 에너지 등 특정 산업에 대한 영향**: 어떤 산업군에 어떤 이슈가 영향을 주었는지 정리하고, 향후 전망을 정량 기반으로 제시하라.

조건:
- **두루뭉술한 표현(예: '불확실성이 커지고 있다', '복합적인 양상')을 피하라.**
- **기사에서 제시된 수치, 기관명, 국가명, 시기 등**을 최대한 인용하라.
- **요약이 아니라 분석을 하라.**, 즉, 단순 나열이 아니라 **인과 관계를 정리하고 시사점까지 서술**하라.
- 문장마다 "무엇이 어떻게 되었다 → 왜 중요한가" 흐름을 유지하라.

아래는 기사 리스트다:


출력 형식 예시:
- [항목 제목]
- [정량적 사실 요약 (무엇이 일어났는가)]
- [그에 따른 시장 반응 (지수, 주가 등)]
- [시사점 및 영향 분석]

아래는 기사 요약이다:


"""),
("placeholder", "{chat_history}"),
("user", "{input}"),
("placeholder", "{agent_scratchpad}")
])
tools = [general_economic_analyze]
agent = create_tool_calling_agent(llm, tools, prompt_template)
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [41]:
agent_executor.invoke({"input": "현재 경제시장이 어떤지 분석해줘."})



> Entering new AgentExecutor chain...

Invoking: `general_economic_analyze` with `{}`


검색 기사 10건:

1. Asian Stocks Steady Near 4-Year High as Tariff Talks and Fed Policy Await
URL: https://www.econotimes.com/Asian-Stocks-Steady-Near-4-Year-High-as-Tariff-Talks-and-Fed-Policy-Await-1716696
본문 요약:
Asian stock markets hovered near four-year highs on Tuesday, supported by Wall Street’s record close, as investors awaited major corporate earnings and developments in U.S. tariff negotiations with key trade partners.

Japan’s markets reopened after a holiday, reacting modestly to weekend election results. Prime Minister Shigeru Ishiba’s ruling coalition lost ground in upper house elections, but Ishiba pledged to stay in office. The yen held steady at 147.46 per dollar after rallying 1% on Monday. Japanese equities opened higher but pared gains, while bond markets remained calm, with most of the political outcome priced in.

Commonwealth Bank economist Kristina Clifton noted that Ishiba’s we

{'input': '현재 경제시장이 어떤지 분석해줘.',
 'output': '## 경제 시장 분석\n\n- **금리 정책 동향**\n    - **호주**: 호주 중앙은행(RBA)은 7월 회의에서 금리 동결을 결정했다. 이는 점진적이고 신중한 완화 전략을 유지하기 위한 결정이며, 9명의 이사 중 6명이 금리 유지를 선택했다. RBA는 추가 금리 인하의 필요성을 인지하고 있지만, 국내 인플레이션과 세계 경제 상황에 대한 불확실성 때문에 신중한 접근 방식을 취하고 있다. 시장은 추가 금리 인하를 예상했지만, RBA는 다음 분기 인플레이션 지표를 포함한 추가 데이터를 기다리기로 했다.\n    - **뉴질랜드**: 뉴질랜드의 2분기 핵심 인플레이션은 2.8%로 소폭 하락했다. 뉴질랜드 통계청에 따르면 헤드라인 인플레이션은 2.7%로 1년 만에 최고치를 기록했다. 전반적인 경제 둔화로 인해 시장은 뉴질랜드 중앙은행(RBNZ)이 금리 인하를 단행할 것이라는 기대감을 재평가하고 있다. RBNZ는 1%~3%의 인플레이션 목표 범위를 유지해야 한다.\n    - **미국**: 연준 이사 크리스토퍼 월러는 7월 말까지 25bp 금리 인하를 촉구했다. 그는 미국 노동 시장의 긴장 징후가 나타나면서 통화 정책을 제한적인 영역에서 중립적인 영역으로 전환해야 한다고 강조했다. 월러는 경제 데이터 둔화를 근거로 제시하며, 트럼프 대통령의 관세가 인플레이션에 미치는 영향은 미미하다고 일축했다.\n    - **영국**: 영란은행(BoE)은 9월 금리 인하 가능성이 낮은 것으로 예상된다. 통화정책위원회(MPC)는 8월과 11월에 금리 인하를 지지할 것으로 예상되며, 이는 전통적인 분기별 간격과 느리고 데이터에 의존적인 접근 방식과 일치한다. 6월에는 금리를 4.25%로 유지했다. 금융 시장과 분석가들은 2025년 말까지 기준 금리가 3.75%가 될 것으로 예상하고 있다.\n\n- **관세 및 무역 정책**\n    - **미국-중국**: 미국 상무부는 중국산 양극재 흑연 수입품에 대해 